# 如何从消息对象中解析文本

:::info 先决条件

本指南假设您熟悉以下概念：
- [聊天模型](/docs/concepts/chat_models/)
- [消息](/docs/concepts/messages/)
- [输出解析器](/docs/concepts/output_parsers/)
- [LangChain Expression Language (LCEL)](/docs/concepts/lcel/)

:::

LangChain 的 [消息](/docs/concepts/messages/) 对象支持 [多种格式](/docs/concepts/messages/#content) 的内容，包括文本、[多模态数据](/docs/concepts/multimodality/) 和一个包含 [内容块](/docs/concepts/messages/#aimessage) 字典的列表。

[聊天模型](/docs/concepts/chat_models/) 响应内容 的格式可能取决于提供商。例如，[Anthropic](/docs/integrations/chat/anthropic/) 的聊天模型会为典型的字符串输入返回字符串内容：

In [1]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-5-haiku-latest")

response = llm.invoke("Hello")
response.content

'Hi there! How are you doing today? Is there anything I can help you with?'

但当生成工具调用时，则会将响应内容结构化为内容块，以传达模型的推理过程：

In [2]:
from langchain_core.tools import tool


@tool
def get_weather(location: str) -> str:
    """Get the weather from a location."""

    return "Sunny."


llm_with_tools = llm.bind_tools([get_weather])

response = llm_with_tools.invoke("What's the weather in San Francisco, CA?")
response.content

[{'text': "I'll help you get the current weather for San Francisco, California. Let me check that for you right away.",
  'type': 'text'},
 {'id': 'toolu_015PwwcKxWYctKfY3pruHFyy',
  'input': {'location': 'San Francisco, CA'},
  'name': 'get_weather',
  'type': 'tool_use'}]

为了自动解析消息对象中的文本，而不考虑底层内容的格式，我们可以使用 [StrOutputParser](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)。我们可以像下面这样将它与聊天模型组合使用：

In [3]:
from langchain_core.output_parsers import StrOutputParser

chain = llm_with_tools | StrOutputParser()

[StrOutputParser](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) 简化了从消息对象中提取文本的操作：

In [4]:
response = chain.invoke("What's the weather in San Francisco, CA?")
print(response)

I'll help you check the weather in San Francisco, CA right away.


这在流式处理的上下文中尤其有用：

In [5]:
for chunk in chain.stream("What's the weather in San Francisco, CA?"):
    print(chunk, end="|")

|I'll| help| you get| the current| weather for| San Francisco, California|. Let| me retrieve| that| information for you.||||||||||

参阅 [API Reference](https://python.langchain.com/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) 获取更多信息。